In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from knowtest.specs.manager import SpecManager
model_spec = {
    "email_assistant": {
        "model": "gpt-3.5-turbo",
        "type": "generation",
        "prompt": [
            {
                "role": "user",
                "content": "You are an email assistant. Help reply to this email: {example}\n"
            }
        ]
    },
    "monologue_summarizer": {
        "model": "gpt-3.5-turbo",
        "type": "generation",
        "prompt": [
            {
                "role": "user",
                "content": "Summarize the monologue to key steps for future use: {example}\n"
            }
        ]
    }
}
SpecManager.add_model_specs(model_spec, overwrite=True)

In [ ]:
generator_specs = {
    "zero_shot_email": "You are a senior researcher. Write one email on '{topic}' to a student:\n",
    "zero_shot_monologue": '''Write a think-aloud monologue of a person searching for an item in a library. The monologue should contain more than 10 sentences. The person should face some challenges when searching.
Scenario: {topic}
Format: [time] sentence
Example: [00:00] Where is the book?'''
}
    
SpecManager.add_generator_specs(generator_specs, overwrite=True)

In [ ]:
from knowtest.server.Capability import Capability

In [ ]:
c = Capability(topic="searching for an item in a library", model_spec="monologue_summarizer", uid="test", generator_specs={"zero_shot_data_gen": "zero_shot_monologue"}, serverHost='172.24.20.95')

In [ ]:
c.display()

In [ ]:
c.change_topic("summarize a think-aloud monologue")